# Hackathon Enedis

Code et Notebook réalisés par : **Tom Le Ber**.

Site web du hackthon : [https://challengedata.ens.fr/participants/challenges/160/](https://challengedata.ens.fr/participants/challenges/160/).

*Vous pouvez également retrouver tout le code écrit dans ce notebook dans les fichiers `solution.py` et `utils.py` de ce même dépot GitHub.*

## Imports des bibliothèques nécessaires pour la bonne exécution de ce notebook :

In [ ]:
%pip install pandas numpy matplotlib scikit-learn joblib tqdm optuna

In [ ]:
from typing import Tuple
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed
from tqdm import tqdm # Sert juste à avoir une représentation visuelle de la progression des boucles
import optuna

SEED = 42

## I. Explication de la méthode de résolution : Imputation par régression avec hyperparamètres optimisés

### A. Régression Ridge

Ridge est un modèle de **régression linéaire** avec une pénalisation $L2$ sur les coefficients.

On cherche un vecteur de paramètres $a \in \mathbb{R}^d$ et un biais $b \in \mathbb{R}$ qui minimisent :

$$
  \mathcal{L}_{\text{ridge}}(a, b) = \frac{1}{n} \sum_{i=1}^n \big( y_i - (a x_i + b) \big)^2 \;+\; \alpha \, \lVert a \rVert_2^2
$$

Où :
- Le premier terme est l'erreur quadratique moyenne ($MSE$),
- Le second terme est la pénalisation $L2$,
- $\alpha > 0$ contrôle la force de la régularisation.

Intuitivement, Ridge :
- Évite que les coefficients deviennent trop grands,
- Améliore la stabilité du modèle quand les features sont corrélées ou bruitées,
- Réduit la variance au prix d'un peu plus de biais.

Il est important de mentionner que Ridge est sensible à l'échelle des variables, une feature en `[0, 1]` et une autre en `[0, 10^6]` seraient pénalisées de façon très différente.

On utilise donc un `StandardScaler` qui effectue, feature par feature, un centrage réduction de moyenne $0$ et d'écart-type $1$ :
$$
  {x_j}_{\text{scaled}} = \frac{x_j - \mu_j}{\sigma_j}
$$

Où :
- Chaque feature correspond à une colonne du tableau de données,
- $\mu_j$ est la moyenne de la feature $j$,
- $\sigma_j$ est l'écart-type de la feature $j$.

Voici un exemple comparant une régression linéaire classique et une régression Ridge avec `alpha=10` sur un jeu de données quasi-linéaire (légèrement bruité) :

In [ ]:
rng = np.random.RandomState(0)
X = rng.randn(100, 1)
y = 3 * X[:, 0] + rng.randn(100) * 0.5

models = {
    "Régression Linéaire": make_pipeline(StandardScaler(), LinearRegression()),
    "Régression Ridge (alpha=10)": make_pipeline(StandardScaler(), Ridge(alpha=10)),
}

xx = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)

plt.scatter(X[:, 0], y, alpha=0.5, color='gray')
for name, model in models.items():
    model.fit(X, y)
    yy = model.predict(xx)
    plt.plot(xx[:, 0], yy, label=name)

plt.legend()
plt.title("Exemple de régression linéaire vs Ridge")
plt.show()

### B. Gradient Boosting sur arbres

Le **Gradient Boosting** construit un modèle comme somme d'arbres de décision :
$$
  F_M(x) = \sum_{m=1}^M \gamma_m \, h_m(x)
$$

Où :
- Chaque $h_m$ est un arbre de décision peu profond,
- Chaque nouvel arbre est entraîné pour corriger les erreurs du modèle précédent,
- On minimise une fonction de perte (ici typiquement la $MSE$) par descente de gradient fonctionnelle.

Intuitivement :
- Le premier arbre capture une tendance grossière,
- Les suivants corrigent progressivement les résidus.

La version `HistGradientBoostingRegressor` de `sklearn` est une implémentation optimisée :
- Les features sont discrétisées en "buckets" (histogrammes),
- Cela accélère énormément la recherche des meilleurs splits,
- Bien adapté aux gros tableaux de données numériques.

Contrairement à Ridge, les arbres ne nécessitent **pas** de standardisation des features, ils capturent naturellement des non-linéarités (seuils, interactions, effets locaux, etc.).

In [ ]:
rng = np.random.RandomState(0)
X = np.linspace(-3, 3, 200).reshape(-1, 1)
y_true = np.sin(X[:, 0])  # vraie fonction
y = y_true + rng.normal(scale=0.3, size=X.shape[0])  # bruit

# Modèle de Gradient Boosting :
gbr = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
)
gbr.fit(X, y)

# Choix de quelques étapes pour visualiser la construction progressive :
stages = [0, 1, 5, 20, 50, 100]
xx = np.linspace(-3, 3, 500).reshape(-1, 1)

# On pré-calcule toutes les prédictions visualisées pour éviter de refaire la boucle à chaque fois :
staged_preds = list(gbr.staged_predict(xx))

plt.figure(figsize=(10, 8))

for index, nb_trees in enumerate(stages, start=1):
    if nb_trees == 0:
        # prédiction du modèle initial (avant tout arbre) :
        y_pred = gbr.init_.predict(xx)
        titre = "0 arbres (modèle initial)"
        label_pred = "Modèle initial (0 arbres)"
    else:
        # prédiction après n_trees arbres
        y_pred = staged_preds[nb_trees - 1]
        titre = f"{nb_trees} arbre{'s' if nb_trees != 1 else ''}"
        label_pred = f"Après {nb_trees} arbre{'s' if nb_trees != 1 else ''}"

    plt.subplot(3, 2, index)
    plt.scatter(X[:, 0], y, s=10, alpha=0.75, color='gray', label="Données bruitées")
    plt.plot(xx[:, 0], np.sin(xx[:, 0]), linewidth=2, label="Vraie fonction")
    plt.plot(xx[:, 0], y_pred, linewidth=2, label=label_pred)
    plt.title(titre)
    plt.legend()

plt.suptitle("Construction progressive du modèle par Gradient Boosting", fontsize=14)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### C. Le modèle final : une combinaison de Ridge et HGBR

Le modèle final, pour un vecteur de features $x$, combine deux prédictions :
- $\hat{y}_{\text{ridge}}(x)$ : une prédiction du modèle `Ridge` (linéaire),
- $\hat{y}_{\tiny{HGBR}}(x)$ : une prédiction du modèle `HistGradientBoostingRegressor` (non linéaire).

La prédiction finale est une moyenne pondérée :
$$
  \hat{y}(x) = w_{\text{ridge}} \, \hat{y}_{\text{ridge}}(x) + \big(1 - w_{\text{ridge}}\big) \, \hat{y}_{\tiny{HGBR}}(x)
$$

Dans mon code, ce poids est contrôlé par `RIDGE_PROPORTION`.

Cotte combinaison mêle les aventages des deux modèles :
- Ridge apporte une tendance globale, plus "lisse" et régulière,
- HGBR apporte de la flexibilité pour capturer les non-linéarités,
- La combinaison permet quasiment tout le temps d'avoir un meilleur résultat que celui obtenu par chaque modèle séparément.

In [73]:
def ensemble_impute_ridge_HGBR(
        x_train: pd.DataFrame,
        y_train: pd.Series,
        x_missing: pd.DataFrame,
        ridge_parameters: dict,
        hgbr_parameters: dict,
        ridge_proportion: float
    ) -> np.ndarray:
    """
    Entraîne les deux modèles donnés ci-dessous et retourne une prédiction combinée, pondérée selon RIDGE_PROPORTION.
        1. Ridge (Linéaire), nécessite une standardisation des données.
        2. HistGradientBoostingRegressor (Non-Linéaire).

    Args:
        x_train (pd.DataFrame): Données d'entraînement (voisins complètes).
        y_train (pd.Series): Cible d'entraînement (valeurs complètes).
        x_missing (pd.DataFrame): Données à prédire (voisins avec trous).
        ridge_parameters (dict): Dictionnaire des hyperparamètres pour le modèle Ridge.
        hgbr_parameters (dict): Dictionnaire des hyperparamètres pour le modèle HGBR.
        ridge_proportion (float): Poids pour la prédiction Ridge dans l'ensemble.
    Returns:
        np.ndarray: Prédictions combinées pour les données manquantes.
    """
    # Création des scalers pour la standardisation des données :
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_missing_scaled = scaler.transform(x_missing)

    # Création et entraînement des modèles :
        # 1. Ridge (Linéaire) :
    ridge_model = Ridge(**ridge_parameters)
    ridge_model.fit(x_train_scaled, y_train)
    ridge_prediction = ridge_model.predict(x_missing_scaled)
        # 2. Gradient Boosting (Non-Linéaire) :
    HGBR_model = HistGradientBoostingRegressor(**hgbr_parameters)
    HGBR_model.fit(x_train, y_train)
    HGBR_prediction = HGBR_model.predict(x_missing)

    # Combinaison des prédictions :
    return (ridge_proportion * ridge_prediction) + ((1 - ridge_proportion) * HGBR_prediction)

*Et il ne nous reste plus "qu'à" déterminer le poids optimal `RIDGE_PROPORTION` par validation croisée...*

### D. Stratégie d’imputation d’une colonne `holed_*`

Pour prédire une colonne cible `holed_k` :
1. On identifie les lignes où la valeur est présente / manquante.
2. On sélectionne parmi `complete_columns` les colonnes prédictives candidates.
3. On calcule la corrélation entre chaque feature candidate et la colonne cible (sur les lignes sans NaN).
4. On garde les `NB_NEIGHBORS` colonnes les plus corrélées en valeur absolue.
5. On entraîne l'ensemble `Ridge` + `HGBR` sur ces lignes complètes et ces colonnes.
6. On prédit les valeurs manquantes et on les insère dans la colonne.

In [ ]:
def impute_column(
        df: pd.DataFrame,
        name_column_to_fill: str,
        complete_columns: list,
        nb_neighbors: int,
        ridge_parameters: dict,
        hgbr_parameters: dict,
        ridge_proportion: float) -> pd.Series:
    """
    Impute les valeurs manquantes dans une colonne cible donnée en utilisant les colonnes complètes spécifiées.
    Utilise une combinaison de Ridge et HistGradientBoostingRegressor pour l'imputation.

    Args:
        df (pd.DataFrame): DataFrame contenant toutes les données.
        name_column_to_fill (str): Nom de la colonne cible à imputer.
        complete_columns (list): Liste des noms de colonnes complètes à utiliser comme prédicteurs.
        nb_neighbors (int): Nombre de voisins les plus corrélés à utiliser pour l'imputation.
        ridge_parameters (dict): Dictionnaire des hyperparamètres pour le modèle Ridge.
        hgbr_parameters (dict): Dictionnaire des hyperparamètres pour le modèle HGBR.
        ridge_proportion (float): Poids pour la prédiction Ridge dans l'ensemble.
    Returns:
        pd.Series: Série contenant la colonne imputée.
    """
    # Suppression des warnings 'inutiles' dans la console :
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    np.seterr(divide='ignore', invalid='ignore')

    # Extraction de la colonne cible et création des masques :
    y_full_input = df[name_column_to_fill]

    mask_missing = y_full_input.isna()
    mask_valid = ~mask_missing

    # Si la colonne est déjà complète, on la retourne telle quelle :
    if not mask_missing.any():
        return y_full_input

    y_valid = y_full_input[mask_valid] # Les valeurs connues à imputer.

    # Sélection des colonnes / features candidates "voisines" de la colonne cible :
    x_candidates = df.loc[mask_valid, complete_columns]

    # Calcul des corrélations entre les colonnes candidates et la colonne cible :
    correlations = x_candidates.corrwith(y_valid)
    correlations = correlations.dropna()

    # Si aucune corrélation n'est trouvée, on remplit avec la moyenne des valeurs connues :
    if correlations.empty:
        fill_value = y_valid.mean()
        y_completed = y_full_input.fillna(fill_value)

    # Sinon, on procède à l'imputation en utilisant les nb_neighbors voisins les plus corrélés :
    else:
        # Sélection des nb_neighbors voisins les plus corrélés (en valeur absolue) :
        top_neighbors = correlations.abs().nlargest(nb_neighbors).index

        # Préparation des données d'entraînement et de prédiction :
        x_train = df.loc[mask_valid, top_neighbors]
        y_train = y_valid
        x_missing = df.loc[mask_missing, top_neighbors]

        # Prédiction des valeurs manquantes :
        y_predicted = ensemble_impute_ridge_HGBR(x_train, y_train, x_missing, ridge_parameters, hgbr_parameters, ridge_proportion)

        # Remplissage des valeurs manquantes dans la série finale :
        y_completed = y_full_input.copy()
        y_completed.loc[mask_missing] = y_predicted

    return y_completed

### E. Imputation de toutes les colonnes `holed_*`

Pour imputer toutes les colonnes `holed_*`, il suffit de répéter la procédure précédente pour chaque colonne cible :
- La fonction `impute_holed_columns` suivante applique `impute_column` à chaque colonne `holed_*`,
- En faisant (éventuellement) tourner en parallèle l'imputation de chaque colonne via `joblib`.

In [ ]:
def impute_holed_columns(
        df: pd.DataFrame,
        columns_to_fill: list[str],
        complete_columns: list[str],
        nb_neighbors: int,
        ridge_parameters: dict,
        hgbr_parameters: dict,
        ridge_proportion: float,
        with_parallelism: bool = False,
        nb_jobs: int = -1
    ) -> pd.DataFrame:
    """
    Impute les colonnes spécifiées dans 'columns_to_fill' en utilisant les colonnes complètes données dans 'complete_columns'.
    Utilise le parallélisme pour (très largement) accélérer le processus.

    Args:
        df (pd.DataFrame): DataFrame contenant toutes les données.
        columns_to_fill (list): Liste des noms de colonnes à imputer.
        complete_columns (list): Liste des noms de colonnes complètes à utiliser comme prédicteurs.
        nb_neighbors (int): Nombre de voisins les plus corrélés à utiliser pour l'imputation.
        ridge_parameters (dict): Dictionnaire des hyperparamètres pour le modèle Ridge.
        hgbr_parameters (dict): Dictionnaire des hyperparamètres pour le modèle HGBR.
        ridge_proportion (float): Poids pour la prédiction Ridge dans l'ensemble.
        with_parallelism (bool): Si vrai, utilise joblib.Parallel pour exécuter les imputations en parallèle.
        nb_jobs (int): Nombre de jobs parallèles à utiliser si with_parallelism est vrai.
    Returns:
        pd.DataFrame: DataFrame contenant les colonnes imputées.
    """
    # Création et exécution des tâches parallèles :
    if with_parallelism:
        results = Parallel(n_jobs=nb_jobs)(
            delayed(impute_column)(df, column, complete_columns, nb_neighbors, ridge_parameters, hgbr_parameters, ridge_proportion)
            for column in tqdm(columns_to_fill)
        )
    else :
        results = [
            impute_column(df, column, complete_columns, nb_neighbors, ridge_parameters, hgbr_parameters, ridge_proportion)
            for column in tqdm(columns_to_fill)
        ]

    # Reconstruction du DataFrame final :
        # Convertion du résultat en un dictionnaire de (column_name, series) :
    result_dict: dict[str, pd.Series] = dict(zip(columns_to_fill, results))
        # Concaténation de toutes les séries prédites :
    result_df = pd.concat(result_dict, axis=1)
        # On restaure l'index d'origine (Horodate) afin de pouvoir exporter le résuktats en un CSV valide :
    result_df.index.name = 'Horodate'
    result_df.reset_index(inplace=True)
        # Reconstruction des colonnes, dans le bon ordre :
    columns_to_keep = ['Horodate'] + [column for column in result_df.columns if 'holed_' in column]

    return result_df[columns_to_keep]

## II. Déterminaison des hyperparamètres

### A. Les différents hyperparamètres

Comme nous l'avons vu, le modèle final dépend de plusieurs hyperparamètres :
- `nb_neighbors` : nombre de colonnes voisines les plus corrélées,
- `ridge_alpha` : régularisation de Ridge,
- `hgbr_iter` : nombre d'itérations (arbres) du HGBR,
- `hgbr_depth` : profondeur maximale des arbres du HGBR,
- `hgbr_lr` : learning rate du HGBR,
- `w_ridge` : poids de Ridge dans l'ensemble.

Or, la performance du modèle dépend fortement de ces hyperparamètres, il est donc essentiel de les choisir judicieusement.

### B. Optuna pour l'optimisation bayésienne des hyperparamètres

Optuna est une bibliothèque d'optimisation d'hyperparamètres qui utilise des techniques bayésiennes pour explorer efficacement l'espace des hyperparamètres.  
Elle construit un modèle probabiliste de la fonction objectif (ici, la performance du modèle en fonction des hyperparamètres) et utilise ce modèle pour sélectionner les hyperparamètres à tester.
Optuna est une des méthodes les plus efficaces pour l'optimisation d'hyperparamètres, dans le cadre de notre projet.

Il nous suffit donc d'implémenter une fonction objectif qui, pour un ensemble d'hyperparamètres donnés (celui ci-dessus) avec des intervalles définis, entraîne le modèle et renvoie la performance ($MAE$) sur un jeu de validation.

In [ ]:
def evaluate_optimization_target(df_x: pd.DataFrame, df_y: pd.DataFrame, name_column_to_fill: str, parameters: dict, complete_columns: list) -> float:
    """
    Evalue une colonne cible spécifique en utilisant les hyperparamètres donnés.
    Retourne le MAE sur les valeurs manquantes uniquement.

    Args:
        df_x (pd.DataFrame): DataFrame avec les colonnes trouées.
        df_y (pd.DataFrame): DataFrame avec les colonnes complètes (vérité terrain).
        name_column_to_fill (str): Nom de la colonne cible à évaluer.
        parameters (dict): Dictionnaire des hyperparamètres à utiliser.
        complete_columns (list): Liste des noms de colonnes complètes à utiliser comme prédicteurs.
    Returns:
        float: MAE calculée sur les positions initialement manquantes.
    """
    # Suppression des warnings 'inutiles' dans la console :
    warnings.filterwarnings("ignore")

    # Extraction de la colonne cible et création des masques :
    y_true_full = df_y[name_column_to_fill]
    y_input_holed = df_x[name_column_to_fill]
    
    mask_missing = y_input_holed.isna()
    mask_valid = ~mask_missing

    # Si la colonne est déjà complète, on retourne 0.0 comme erreur :
    if not mask_missing.any(): 
        return 0.0

    y_valid = y_input_holed[mask_valid] # Les valeurs connues à imputer.

    # Sélection des colonnes / features candidates "voisines" de la colonne cible :
    X_candidates = df_x.loc[mask_valid, complete_columns]

    # Calcul des corrélations entre les colonnes candidates et la colonne cible :
    corrs = X_candidates.corrwith(y_valid).abs()
    top_k = corrs.nlargest(parameters['nb_neighbors']).index

    # Préparation des données d'entraînement et de prédiction :
    x_train = df_x.loc[mask_valid, top_k]
    y_train = y_valid
    x_missing = df_x.loc[mask_missing, top_k]

    # Calcul des prédictions avec l'ensemble Ridge + HGBR :
    y_predicted = ensemble_impute_ridge_HGBR(
        x_train, y_train, x_missing,
        ridge_parameters = {
            'alpha': parameters['ridge_alpha'],
            'random_state': SEED
        },
        hgbr_parameters = {
            'max_iter': parameters['hgbr_iter'],
            'max_depth': parameters['hgbr_depth'],
            'learning_rate': parameters['hgbr_lr'],
            'early_stopping': True,
            'random_state': SEED
        },
        ridge_proportion = parameters['w_ridge']
    )

    # Calcul du MAE sur les positions initialement manquantes :
    y_true_missing = y_true_full[mask_missing]
    return np.mean(np.abs(y_true_missing - y_predicted)) # Pas exactement la même chose que dans utils.get_metrics, mais BEAUCOUP plus rapide.

In [ ]:
def find_optimized_parameters(
        nb_holed_targets_for_optimization: int,
        nb_trials: int,
        input_file: str = 'data/datasets/x_train.csv',
        output_file: str = 'data/datasets/y_train_true.csv',
        with_parallelism: bool = False,
        nb_jobs: int = -1
    ) -> dict[str, float]:
    """
    Utilise Optuna pour trouver les hyperparamètres optimaux pour l'imputation des colonnes trouées.
    Évalue les performances sur un sous-ensemble de colonnes cibles pour accélérer le processus.

    Args:
        nb_holed_targets_for_optimization (int): Nombre de colonnes 'holed_*' à échantillonner pour l'optimisation des hyperparamètres (parmi les 1000 disponibles).
        nb_trials (int): Nombre de trials pour l'optimisation des hyperparamètres avec Optuna.
        input_file (str):
            Chemin vers le fichier CSV d'entrée avec les colonnes trouées.
            Par défaut 'data/datasets/x_train.csv' (x_train.csv sur le site web).
        output_file (str):
            Chemin vers le fichier CSV de vérité terrain avec les colonnes complètes.
            Par défaut 'data/datasets/y_train_true.csv' (y_train.csv sur le site web).
        with_parallelism (bool): Si vrai, utilise joblib.Parallel pour accélérer le processus d'évaluation.
        nb_jobs (int): Nombre de jobs parallèles à utiliser si with_parallelism est vrai.
    Returns:
        dict[str, float]: Dictionnaire des meilleurs hyperparamètres trouvés.
    """
    # Ouverture des fichiers de données (et préparation des DataFrames) :
    print(f"Chargement des données pour l'optimisation des hyperparamètres.")

    df_x = pd.read_csv(input_file)
    df_y = pd.read_csv(output_file)

    df_x.set_index('Horodate', inplace=True)
    df_y.set_index('Horodate', inplace=True)

    # Sélection des colonnes cibles et prédicteurs :
    holed_to_predict = [
        column for column in df_x.columns if 'holed_' in column
    ]

    # Extration des colonnes d'entrainement (complètes et non-constantes) :
    potential_predictors = [column for column in df_x.columns if 'holed_' not in column]
    stds = df_x[potential_predictors].std()
    complete_columns = stds[stds > 1e-9].index.tolist() # On enlève les colonnes constantes car elles sont (quasiment) inutiles pour Ridge et HGBR.

    # Choix d'un sous-ensemble aléatoire de nb_holed_targets_for_optimization colonnes cibles (parmis les 1000 'holed_*' pour l'optimisation (trop lent sinon) :
    np.random.seed(SEED)
    subset_targets = np.random.choice(holed_to_predict, size=nb_holed_targets_for_optimization, replace=False)

    print(f"\nRecherche des hyperparamètres les plus optimisés ({nb_trials} trials) :\n\t- Nombre de colonnes 'holed_*' échantillonnées\t = {len(subset_targets)}\n\t- Nombre de colonnes prédictrices\t\t = {len(complete_columns)}\n")

    # Définition de la fonction 'objective' d'Optuna :
    def objective(trial: optuna.trial.Trial) -> float:
        parameters = {
            'nb_neighbors': trial.suggest_int('nb_neighbors', 0, 10000),
            'ridge_alpha':  trial.suggest_float('ridge_alpha', 1.0, 500.0, log=True),
            'hgbr_iter':    trial.suggest_int('hgbr_iter', 100, 500),
            'hgbr_depth':   trial.suggest_int('hgbr_depth', 4, 15),
            'hgbr_lr':      trial.suggest_float('hgbr_lr', 0.01, 0.3),
            'w_ridge':      trial.suggest_float('w_ridge', 0.0, 1.0) 
        }

        if with_parallelism:
            scores = Parallel(n_jobs=nb_jobs)(
                delayed(evaluate_optimization_target)(df_x, df_y, column, parameters, complete_columns)
                for column in subset_targets
            )
        else:
            scores = [
                evaluate_optimization_target(df_x, df_y, column, parameters, complete_columns)
                for column in subset_targets
            ]

        return np.mean(scores)

    # Lancement de l'optimisation avec Optuna :
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=nb_trials)

    # Résultat final :
    print("\nOptimisation complète, meilleurs hyperparamètres trouvés :\n" + f"\n\t".join([f'- {key} = {value}' for key, value in study.best_params.items()]))
    return study.best_params

### C. Exemple d'exécution de l'optimisation des hyperparamètres

In [ ]:
find_optimized_parameters(
    nb_holed_targets_for_optimization = 10,
    nb_trials = 20
)

### D. Hyperparamètres optimisés

Après exécution de la fonction ci-dessus pour $100$ `holed_*` sur $250$ essais, on obtient les hyperparamètres optimaux suivants :

In [ ]:
NB_NEIGHBORS = 10000 # Plus il y a de voisins, plus le modèle peut capturer des relations complexes.

RIDGE_PROPORTION = 0.125

RIDGE_PARAMETERS = {
    'alpha': 10,
    'random_state': SEED
}

HGBR_PARAMETERS = {
    'max_iter': 250,
    'max_depth': 8,
    'learning_rate': 0.2,
    'early_stopping': True,
    'random_state': SEED
}

## III. Exécution finale

### A. Fonctions auxilliaires

On introduit les fonctions suivantes afin :
- De pouvoir estimer la $MAE$ et le $R^2$ d'un modèle d'imputation entrainé sur les colonnes `holed_*` du dataset `x_train.csv` (dont on connaît les vraies valeurs avec `y_train.csv`),
  - Ceci permet de juger de la qualité de notre imputation avant de l'appliquer sur le dataset de test.
- Et de pouvoir lire les fichiers CSV donnés, en fixant la colonne `Horodate` comme index.

In [76]:
def get_metrics(holed_full: pd.DataFrame, holed_predicted: pd.DataFrame, holed_nans: pd.DataFrame) -> Tuple[float, float]:
    """
    Calcule la MAE et le R² pour un ensemble de colonnes trouées données, uniquement sur les positions contenant des trous (NaNs) à l'origine.
    Ignore les positions où holed_full ou holed_predicted sont NaN.

    Args:
        holed_full (pd.DataFrame): DataFrame avec les vraies valeurs complètes.
        holed_predicted (pd.DataFrame): DataFrame avec les valeurs prédites.
        holed_nans (pd.DataFrame): DataFrame avec les trous (NaNs) d'origine.
    Raises:
        ValueError: Si les shapes des DataFrames ne correspondent pas.
    Returns:
        Tuple[float, float]: MAE et R² calculés sur les positions initialement manquantes.
    """
    # On s'assure que les shapes sont compatibles :
    if holed_full.shape != holed_predicted.shape or holed_full.shape != holed_nans.shape:
        raise ValueError(
            f"Shapes incompatibles :\n\t- holed_full = {holed_full.shape}\n\t- holed_predicted = {holed_predicted.shape}\n\t- holed_nans = {holed_nans.shape}"
        )

    # On masque des trous d'origine :
    mask_missing_values = holed_nans.isna().values # Renvoie un tableau 2D de booléens.

    # On récupère les valeurs réelles et prédites :
    true_holed = holed_full.values
    predicted_holed = holed_predicted.values

    # On liste les positions de chaque valeur prédite et si aucune n'est valide, on retourne (-1.0, 0.0) :
    valid_mask = (
        mask_missing_values
        & np.isfinite(true_holed)
        & np.isfinite(predicted_holed)
    )
    if valid_mask.sum() == 0:
        return -1.0, 0.0

    # On extrait les valeurs valides :
    true_values = true_holed[valid_mask]
    pred_values = predicted_holed[valid_mask]

    # Calcul de la MAE :
    diff = np.abs(true_values - pred_values)
    mae = float(diff.mean())

    # Calcul du R² :
    ss_res = float(np.sum((true_values - pred_values) ** 2))
    ss_tot = float(np.sum((true_values - true_values.mean()) ** 2))
    r2 = 1 - ss_res / ss_tot if ss_tot != 0 else 0.0

    return mae, r2

In [77]:
def get_dataframe(path: str) -> pd.DataFrame:
    """
    Lit le fichier CSV donné et retourne un DataFrame pandas de ce fichier avec la colonne 'Horodate' en tant qu'index.

    Args:
        path (str): Chemin vers le fichier CSV à lire.
    Raises:
        ValueError: Si la colonne 'Horodate' est absente du CSV.
    Returns:
        pd.DataFrame: DataFrame pandas avec les données du CSV, sans la colonne 'Horodate'.
    """
    df = pd.read_csv(path)

    if 'Horodate' not in df.columns:
        raise ValueError("Le fichier CSV doit contenir une colonne 'Horodate'.")

    df.set_index('Horodate', inplace=True)

    return df

### B. Fonction principale d’imputation avec hyperparamètres optimisés

On définit la fonction `run_imputation` qui :
- Charge le fichier d'entrée (train ou test),
- Identifie les colonnes `holed_*`,
- Supprime les features constantes,
- Lance `impute_holed_columns`,
- Sauvegarde les prédictions,
- Et, si on est en mode train, calcule MAE et R² avec `get_metrics`.

In [ ]:
def run_imputation(
        nb_neighbors: int,
        ridge_parameters: dict,
        hgbr_parameters: dict,
        ridge_proportion: float,
        train_mode: bool = False,
        input_file: str = None,
        output_file: str = None
    ) -> None:
    """
    Fonction principale pour exécuter l'imputation des colonnes trouées dans un fichier donné.
    Charge les données, effectue l'imputation et sauvegarde les résultats dans un fichier CSV.

    Args:
        nb_neighbors (int): Nombre de voisins les plus corrélés à utiliser pour l'imputation.
        ridge_parameters (dict): Dictionnaire des hyperparamètres pour le modèle Ridge.
        hgbr_parameters (dict): Dictionnaire des hyperparamètres pour le modèle HGBR.
        ridge_proportion (float): Poids pour la prédiction Ridge dans l'ensemble.
        train_mode (bool):
            Si vrai, réalise une prédiction sur le dataset de train et renvoie le MAE & R² associée.
            Si faux, réalise une prédiction sur le dataset de test.
        input_file (str):
            Chemin vers le fichier d'entrée contenant les données avec trous.
            Par défaut 'data/datasets/x_train.csv' si train_mode == True, sinon 'data/datasets/x_test.csv'.
        output_file (str):
            Chemin vers le fichier de sortie pour enregistrer les données imputées.
            Par défaut 'y_train.csv' si train_mode == True, sinon 'y_test.csv'.
    """
    # Sélection des fichiers d'entrée et de sortie en fonction du mode (train/test) :
    if input_file is None:
        input_file = 'data/datasets/x_train.csv' if train_mode else 'data/datasets/x_test.csv'
    if output_file is None:
        output_file = 'y_train.csv' if train_mode else 'y_test.csv'

    # Chargement du fichier d'entrée :
    print(f"Chargement du fichier '{input_file}' :")
    df = get_dataframe(input_file)

    # Extraction des colonnes à prédire 'holed_*' :
    holed_to_predict = [
        column for column in df.columns if 'holed_' in column
    ]
    if not holed_to_predict:
        raise ValueError("Aucune colonne cible 'holed_' trouvée dans le fichier d'entrée.")

    # Extration des colonnes d'entrainement (complètes et non-constantes) :
    potential_predictors = df.columns.difference(holed_to_predict)
    stds = df[potential_predictors].std()
    complete_columns = stds[stds > 1e-9].index.tolist() # On enlève les colonnes constantes car elles sont (quasiment) inutiles pour Ridge et HGBR.

    print(f"\t- Colonnes à prédire (holed)\t = {len(holed_to_predict)}\n\t- Colonnes d'entraînement\t = {len(complete_columns)}")

    # Lancement de l'imputation des données :
    print(f"\nImputation de {len(holed_to_predict)} colonnes :")
    holed_predicted = impute_holed_columns(df, holed_to_predict, complete_columns, nb_neighbors, ridge_parameters, hgbr_parameters, ridge_proportion)
    # Sauvegarde du résultat :
    print(f"\nEnregistrement des prédictions dans '{output_file}'.")
    holed_predicted.to_csv(output_file, index=False)

    # Si on est en mode train (utilise le dataset x_train), on affiche la MAE et le R² associé :
    if train_mode:
        holed_true = get_dataframe('data/datasets/y_train_true.csv').filter(like='holed_')
        holed_nans = get_dataframe('data/datasets/x_train.csv').filter(like='holed_')

        mae, r2 = get_metrics(holed_true, holed_predicted.filter(like='holed_'), holed_nans)
        print(f"\nMétriques [MODE TRAIN] :\n\t- MAE\t = {mae}\n\t- R2\t = {r2}")

### C. Exécution complète de l’imputation

Il ne nous reste plus qu'à appeler la précédente fonction :

In [ ]:
TRAIN_MODE = False # Si vrai, réalise une prédiction sur le dataset de train et renvoie le MAE & R² associée. Si faux, réalise une prédiction sur le dataset de test.

run_imputation(
    nb_neighbors = NB_NEIGHBORS,
    ridge_parameters = RIDGE_PARAMETERS,
    hgbr_parameters = HGBR_PARAMETERS,
    ridge_proportion = RIDGE_PROPORTION,
    train_mode = TRAIN_MODE
)

## Références

### Régression Ridge
- Documentation scikit-learn, `sklearn.linear_model.Ridge` (référence API)  
  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html  
- Guide utilisateur scikit-learn — *1.1. Modèles linéaires : régression Ridge*  
  https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression

### Histogram-based Gradient Boosting Regressor (HGBR)
- Documentation scikit-learn, `sklearn.ensemble.HistGradientBoostingRegressor` (référence API)  
  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html  
- Guide utilisateur scikit-learn — *1.11. Méthodes d’ensemble : Histogram-Based Gradient Boosting*  
  https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting  
- YouTube, *Hist Gradient Boosting Regression*  
  https://www.youtube.com/watch?v=V3pXpCnLgqw

### Optuna (optimisation d’hyperparamètres)
- Site officiel, Optuna : framework d’optimisation d’hyperparamètres  
  https://optuna.org/  
- Documentation officielle et tutoriels (Read the Docs)  
  https://optuna.readthedocs.io/en/stable/